In [ ]:
# Install required libraries
!pip install transformers datasets peft accelerate evaluate scikit-learn

In [ ]:
#Install and use wandb for performing a Hyper-parameter sweep
!pip install -U wandb --quiet          # if not already installed
import wandb
wandb.login()                          # paste the API key when asked

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
import pandas as pd
from datasets import Dataset
df = pd.read_json("hr_sentiment_starter.jsonl", lines=True)
dataset = Dataset.from_pandas(df)
label_map = {
    "highly engaged": 0,
    "content": 1,
    "disengaged": 2,
    "at risk of leaving": 3
}

dataset = dataset.map(lambda x: {"label": label_map[x["label"]]})

dataset = dataset.train_test_split(test_size=0.2)
dataset['validation'] = dataset.pop('test')

In [ ]:
# Initialize model
from transformers import AutoModelForSequenceClassification
model_name="j-hartmann/emotion-english-distilroberta-base"

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4,
        ignore_mismatched_sizes=True
    )

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Tokenize dataset
def tokenize(example):
    return tokenizer(example['pros_cons'], padding='max_length', truncation=True)

tokenized = dataset.map(tokenize, batched=True)

In [ ]:
sweep_config = {
    "method": "grid",          # or "random", "bayes"
    "metric": {
        "name": "eval_f1",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate":  {"values": [2e-5, 5e-5]},
        "per_device_train_batch_size": {"values": [8,16]},
        "num_train_epochs": {"values": [3,4,5]}
    }
}
sweep_id = wandb.sweep(sweep_config, project="hr-sentiment-sweep-v2")


In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

def sweep_train():
    # Fetch current sweep hyper-params from wandb.config
    run   = wandb.init()            # project/name is inherited from the sweep
    config = run.config

    args = TrainingArguments(
        output_dir="./results/sentiment-model",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        learning_rate=config.learning_rate,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=config.per_device_train_batch_size,
        num_train_epochs=config.num_train_epochs,
        report_to="wandb",
        run_name=run.name
    )

    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    # Save model after training
    model_dir = f"./results/model-{run.id}"
    trainer.save_model(model_dir)

    # At the end, log classification report on validation set
    preds_output = trainer.predict(tokenized["validation"])
    preds  = preds_output.predictions.argmax(-1)
    labels = preds_output.label_ids

    from sklearn.metrics import classification_report
    report = classification_report(
        labels, preds,
        target_names=list(label_map.values()),
        output_dict=True
    )
    wandb.log({"classification_report": report})
    # Log the artifact
    artifact = wandb.Artifact(f"model-{wandb.run.name}", type="model")
    artifact.add_dir("./results/sentiment-model")  # This is where Trainer saved the model
    wandb.log_artifact(artifact)
    run.finish()


In [ ]:
wandb.agent(sweep_id, function=sweep_train)

In [ ]:
# Use this if you want to load the model from artifact
import wandb

# Log in first
wandb.login()

# Restore the best model from W&B
run = wandb.init(project="hr-sentiment-sweep-v2", id="q8smpdk6", resume="allow")

artifact = run.use_artifact("model-worthy-sweep-11:v0")  # or "model:v0" if named that way
model_dir = artifact.download()


In [ ]:
# Use this if you want to load the model from local
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_dir = "./results/model-q8smpdk6"
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)


In [ ]:
from transformers import DataCollatorWithPadding
import pandas as pd
from datasets import Dataset

df = pd.read_json("employee_reviews_testset_100_each.jsonl", lines=True)
test_dataset = Dataset.from_pandas(df)

test_dataset = test_dataset.map(lambda x: {"label": label_map[x["label"]]})

test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, compute_metrics=compute_metrics)
results = trainer.evaluate(test_dataset)
print(results)
